<a href="https://colab.research.google.com/github/ninadcpa/DSC540/blob/main/Week_7_8_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSC540 Week 7 and 8 Exercise 
@author Ninad Patkhedkar

In [1]:
# This code is written for mounting code folder checked out from github on google drive
# All assignment code will be maintained inside folder "DSC540"
# Code from book is inside folder "Data-Wrangling-with-Python"

from google.colab import drive
import sys
import os

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DSC540')
os.chdir('/content/drive/MyDrive/Colab Notebooks/DSC540')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The four chapters you read these two weeks focus extensively on cleaning and transforming data.

I am choosing below dataset for this exercise

*  [So Much Data Candy, Seriously (Ng, 2017)](https://www.scq.ubc.ca/wp-content/uploads/2017/10/candyhierarchy2017.csv) 

I have downloaded the data and will use local file.


In [2]:
import numpy as np
import pandas as pd

# File was throwing some encoding error while trying to read with default utf-8 encoding
# Using latin-1 encoding to fix issue
# Missing values or spaces would be replaced by np.nan
na_values = [""," ",None,"Missing","NA"]
df=pd.read_csv("candyhierarchy2017.csv",encoding='latin-1',na_values=na_values)
df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we donÕt know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,...,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | ReeseÕs Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,...,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,...,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,

Perform 2 tasks out of below list

Chapter 7
* Filter out missing data
* Fill in missing data
* Remove duplicates
* Transform data using either mapping or a function
* Replace values
* Discretization and Binning
* Manipulate Strings
* Noticed that missing Values are automatically populated as NaN



In [3]:
df.shape

(2460, 120)

In [4]:
df.isna().sum()

Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64

Internal ID value is unique. I will use Interanl ID value as index.


In [5]:
df.set_index(['Internal ID'],inplace=True)
df.head()

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we donÕt know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | HersheyÕs Milk Chocolate,...,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | ReeseÕs Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
Internal ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,...,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,MEH,...,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DES

There are lot of missing values.
We will delete all rows for which all values are missing i.e. NaN

In [6]:
df.dropna(axis=0,how='all',inplace=True)
df.shape

(2439, 119)

It shows that around 21 rows dropped as it didn't had any value populated.

Check if there are any duplicate records.

In [7]:
df.duplicated().sum()

15

Drop duplicates.

In [8]:
df.drop_duplicates(inplace=True)
df.shape

(2424, 119)

There are few columns for which more than 50% records have missing values i.e. NaN.

I don't think those columns would be of any use, hence dropping those columns.

In [9]:
df.dropna(thresh=df.shape[0]*0.5,how='all',axis=1,inplace=True)
df.shape

(2424, 112)

Notice that 7 columns are dropped.

Country names are missing at some places and also in different cases like US, us and some places usa

In [10]:
df['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'Mexico', 'United states', 'u.s.a.', 'USAUSAUSA', 'america', '35',
       'france', 'United States of America ', 'U.S.A.', 'finland',
       'unhinged states', 'Canada ', 'united states of america',
       'US of A', 'Unites States', 'The United States', 'North Carolina ',
       'Unied States', 'Netherlands', 'germany', 'Europe', 'Earth', 'U S',
       'u.s.', 'U.K. ', 'Costa Rica', 'The United States of America',
       'unite states', 'U.S.', '46', 'cascadia', 'Australia',
       'insanity lately', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", 'usas', 'Pittsburgh', '45', 'United State', '32',
       'France', 'australia', 'A', 'Can', 'Canae', 'New York',
       'Trumpistan', 'Ireland', 'United Sates', '

There are too many 129 countries with varying names. I will just try to correct country name for US.

First changing all contry names to uppercase.

In [11]:
df['Q4: COUNTRY'] = df['Q4: COUNTRY'].str.upper()

In [12]:
df['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'US', nan, 'CANADA', 'MURICA', 'UNITED STATES',
       'UK', 'UNITED KINGDOM', 'UNITED STATES ', 'UNITED STAES',
       'UNITED STATES OF AMERICA', 'UAE', 'ENGLAND', 'CANADA ', 'MEXICO',
       'U.S.A.', 'USAUSAUSA', 'AMERICA', '35', 'FRANCE',
       'UNITED STATES OF AMERICA ', 'FINLAND', 'UNHINGED STATES',
       'US OF A', 'UNITES STATES', 'THE UNITED STATES', 'NORTH CAROLINA ',
       'UNIED STATES', 'NETHERLANDS', 'GERMANY', 'EUROPE', 'EARTH', 'U S',
       'U.S.', 'U.K. ', 'COSTA RICA', 'THE UNITED STATES OF AMERICA',
       'UNITE STATES', '46', 'CASCADIA', 'AUSTRALIA', 'INSANITY LATELY',
       'GREECE', 'USA? HARD TO TELL ANYMORE..', "'MERICA", 'USAS',
       'PITTSBURGH', '45', 'UNITED STATE', '32', 'A', 'CAN', 'CANAE',
       'NEW YORK', 'TRUMPISTAN', 'IRELAND', 'UNITED SATES', 'KOREA',
       'CALIFORNIA', 'JAPAN', 'SOUTH AFRICA',
       'I PRETEND TO BE FROM CANADA, BUT I AM REALLY FROM THE UNITED STATES.',
       'ICELAND', 'CANADA`', 'SCOTLAND', 'UK

In [13]:
df['Q4: COUNTRY'].unique().size

105

Now will correct name for country of USA

In [14]:
usa_names_map = {
    'USA': 'USA',
    'USA ': 'USA',
    'US': 'USA',
    'UNITED STATES': 'USA',
    'U.S.': 'USA',
    'UNITED STATES OF AMERICA': 'USA',
    'US OF A': 'USA',
    'U.S.A.': 'USA',
    'USAUSAUSA': 'USA',
    'THE UNITED STATES':'USA',
    'UNIED STATES': 'USA',
    'USA! USA! USA!':'USA',
    'USAA': 'USA',
    'AMERICA': 'USA',
    'UNHINGED STATES': 'USA',
    'THE UNITED STATES OF AMERICA': 'USA', 
    'UNITE STATES': 'USA',
    'UNITED SATES': 'USA',
    'UNITES STATES': 'USA',
    'UNITED STSTES': 'USA',
    'UNITED STATES ': 'USA',
    'UNITED STATES OF AMERICA ': 'USA',
    'U S': 'USA',
    'UNITED STATED': 'USA',
    'USA USA USA!!!!': 'USA'
}

df['Q4: COUNTRY'] = df['Q4: COUNTRY'].map(usa_names_map).fillna(df['Q4: COUNTRY'])

In [15]:
df['Q4: COUNTRY'].unique()

array(['USA', nan, 'CANADA', 'MURICA', 'UK', 'UNITED KINGDOM',
       'UNITED STAES', 'UAE', 'ENGLAND', 'CANADA ', 'MEXICO', '35',
       'FRANCE', 'FINLAND', 'NORTH CAROLINA ', 'NETHERLANDS', 'GERMANY',
       'EUROPE', 'EARTH', 'U.K. ', 'COSTA RICA', '46', 'CASCADIA',
       'AUSTRALIA', 'INSANITY LATELY', 'GREECE',
       'USA? HARD TO TELL ANYMORE..', "'MERICA", 'USAS', 'PITTSBURGH',
       '45', 'UNITED STATE', '32', 'A', 'CAN', 'CANAE', 'NEW YORK',
       'TRUMPISTAN', 'IRELAND', 'KOREA', 'CALIFORNIA', 'JAPAN',
       'SOUTH AFRICA',
       'I PRETEND TO BE FROM CANADA, BUT I AM REALLY FROM THE UNITED STATES.',
       'ICELAND', 'CANADA`', 'SCOTLAND', 'UK ', 'DENMARK', 'FRANCE ',
       'SWITZERLAND', 'AHEM....AMERCA', 'UD', 'SCOTLAND ', 'SOUTH KOREA',
       'NEW JERSEY', 'INDONESIA', 'THE NETHERLANDS', 'UNITED STATSS',
       'ENDLAND', 'ATLANTIS', 'MURRIKA', 'ALASKA', 'SOVIET CANUCKISTAN',
       'N. AMERICA', 'SINGAPORE', 'USSA', 'CHINA', 'TAIWAN', 'IRELAND ',
       'HONG KO

In [16]:
df['Q4: COUNTRY'].unique().size

81

Oops , it looks like too many corrections required for fixing country name. I won't fix it anymore as its taking lot of time. I am not sure if there is any better way to do this.

On side note, I think survey form should have dropdown rather than open text field for country. This can help with standardization and cleaner data collection.

## Chapter 8
* Create hierarchical index
* Combine and Merge Datasets (you will have to either create a new dataset from * your existing data or create a relationship between the data I have provided)
* Reshape
* Pivot the data

In [17]:
candy_names = {x:x.split('Q6 | ')[1] for x in df.columns if x.startswith('Q6 | ')}

In [18]:
#candy_names = [ {x:x.split('Q6 | ')[1]} for x in df.columns if x.startswith('Q6 | ')] 
#candy_names
df.rename(columns=candy_names,inplace=True)
df.reset_index(inplace=True)
#df.set_index(list(candy_names.values()),inplace=True)
#df.head()

In [19]:
df.dropna(subset=['Q2: GENDER','Q3: AGE'],inplace=True)
df.shape

(2363, 113)

### Created Hierachial Data using columns Gender and Age

In [20]:
df.set_index(['Q2: GENDER','Q3: AGE'],inplace=True)
df.head()

Internal ID  ... Click Coordinates (x, y)
Q2: GENDER Q3: AGE               ...                         
Male       44          90272821  ...                 (84, 25)
           49          90272829  ...                      NaN
           40          90272840  ...                 (75, 23)
           23          90272841  ...                 (70, 10)
           53          90272853  ...                      NaN

[5 rows x 111 columns]

In [21]:
#df.reset_index(drop=True,inplace=True)
#df.set_index(['Internal ID'],inplace=True)

In [22]:
new_index = pd.Index(list(candy_names.values()),name='candy')
new_df = df.reindex(columns=new_index)
new_df.head()

candy              100 Grand Bar  ... York Peppermint Patties
Q2: GENDER Q3: AGE                ...                        
Male       44                MEH  ...                 DESPAIR
           49                NaN  ...                     NaN
           40                MEH  ...                 DESPAIR
           23                JOY  ...                     JOY
           53                NaN  ...                     NaN

[5 rows x 103 columns]

### Reshaping data using stack() method

In [23]:
ldata = new_df.stack().reset_index().rename(columns={0: 'candy_liking'})
ldata[:10]

,Q2: GENDER,Q3: AGE,candy,candy_liking
0,Male,44,100 Grand Bar,MEH
1,Male,44,Anonymous brown globs that come in black and o...,DESPAIR
2,Male,44,Any full-sized candy bar,JOY
3,Male,44,Black Jacks,MEH
4,Male,44,Bonkers (the candy),DESPAIR
5,Male,44,Bonkers (the board game),DESPAIR
6,Male,44,Bottle Caps,DESPAIR
7,Male,44,Box'o'Raisins,DESPAIR
8,Male,44,Broken glow stick,DESPAIR
9,Male,44,Butterfinger,DESPAIR
